#### Import Libs

In [289]:
import os
import re

####  File Utils

In [290]:
def read_file(folder, filename):
    filepath = os.path.join(folder, filename)
    file = open(filepath, 'r')
    page = file.read()
    return page

#### Page Rank

In [392]:
def open_pages(folder):
    filenames = os.listdir(folder)
    page_names = list(map(lambda filename: filename.split('.')[0], filenames))
    pages = []
    for filename in filenames:        
        page = read_file(folder, filename)
        pages.append(page)
    return (pages, page_names)

def page_rank_row(page, page_names):
    # n pages
    n = len(page_names)
    
    # search links in the page
    regex_links = '<a href="http://(.*?).html'
    matchs = re.findall(regex_links, page)
    
    # probability 
    p = 1/n if len(matchs) == 0 else 1/len(matchs)
    
    # if pagename exist in the page
    ### that position in the array will be a number > 0
    # else
    ### that position in the array will be a number 0
    row = []
    
    if len(matchs) == 0:
        row = [1/n, 1/n, 1/n, 1/n, 1/n, 1/n]
    else:
        for page_name in pages_names:
            row.append(p if page_name in matchs else 0)
    
    return row

def page_rank(pages, page_names):
    matrix = np.zeros(shape=(len(page_names), len(page_names)))
    for index, page in enumerate(pages):
        row = page_rank_row(page, page_names)
        matrix[index] = row
    return matrix

#### Run

In [393]:
folder = 'paginas'
pages, page_names = open_pages(folder)
matrix = page_rank(pages, page_names)

print(matrix)

[[0.         0.         1.         0.         0.         0.        ]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.         0.         0.         0.         0.33333333 0.33333333]
 [0.         0.         0.         0.5        0.         0.5       ]
 [0.         0.25       0.25       0.         0.25       0.        ]]


#### Ejemplo de como funciona 

In [385]:
page = read_file(folder, "jonvoight.html")
page

'<html>\n<head>\n    <title>Jon Voight</title>\n</head>\n<body>\n    Jonathan Vincent "Jon" Voight (born December 29, 1938) is an American film and television actor. He has had a long and distinguished career as both a leading man and, in recent years, a character actor, with an extensive and compelling range. He came to prominence at the end of the 1960s, with a performance as a would-be hustler in 1969\'s Best Picture winner, Midnight Cowboy, for which he earned his first Academy Award nomination. Throughout the following decades, Voight built his reputation with an array of challenging roles and has appeared in such landmark films as Deliverance (1972), and Coming Home (1978), for which he received an Academy Award for Best Actor. Voight\'s impersonation of sportscaster/journalist Howard Cosell, in the 2001 biopic Ali, earned Voight critical raves and his fourth Oscar nomination. He has starred in the seventh season of 24 as the villain Jonas Hodges.\n    Voight is the father of act

Vemos que tiene 2 links hacia "angelinajolie" y 1 a "bradpitt", entonces:

p sera 1 / 3 => 0.33

Y el array sera cero en las posiciones que no son estas dos pages, y 0.33 en las que si son.

In [361]:
page_rank_row(page, page_names)

[0, 0, 0, 0, 0.3333333333333333, 0.3333333333333333]